In [0]:
import tensorflow as tf
# import cv2 as cv
import os
import numpy as np
import glob
# # from random import shuffle
from google.colab import drive
import matplotlib.pyplot as plt
import csv
import re
import json
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding,Input
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Bidirectional,LSTM,GlobalMaxPool1D,Dense
from tensorflow.contrib import rnn
from sklearn.utils import shuffle
import pandas as pd
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
import statistics as st


Using TensorFlow backend.


In [0]:
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_feed_image=np.load("/content/drive/My Drive/Copy of image_embedding.npy")[1:]

In [0]:
ar=['0ZOPH', 'T7449', 'BA6DD', 'OVHFT', 'SH20Q', '5L04F', 'O2VP8', 'GPR89', '9EEGQ', 'PWR6D', '0RP6B', '0OP1K']

In [0]:
df=pd.read_csv("/content/drive/My Drive/Charades_v1_train.csv")

In [0]:
df.head()

,id,subject,scene,quality,relevance,verified,script,objects,descriptions,actions,length
0,46GP8,HR43,Kitchen,6.0,7.0,Yes,A person cooking on a stove while watching som...,food;stove;window,A person cooks food on a stove before looking ...,c092 11.90 21.20;c147 0.00 12.60,24.83
1,N11GT,0KZ7,Stairs,6.0,7.0,Yes,"One person opens up a folded blanket, then sne...",blanket;broom;floor,Person at the bottom of the staircase shakes a...,c098 8.60 14.20;c075 0.00 11.70;c127 0.00 15.2...,18.33
2,0IH69,6RE8,Bedroom,6.0,5.0,Yes,A person is seen leaving a cabinet. They then ...,book;box;cabinet;shelf,A person is standing in a bedroom. They walk o...,NaN,30.25
3,KRF68,YA10,Laundry room,6.0,7.0,Yes,A person runs into their laundry room. They gr...,clothes;door;phone,A person runs in and shuts door. The person gr...,c018 22.60 27.80;c141 4.10 9.60;c148 10.30 25....,30.33
4,MJO7C,6RE8,Kitchen,6.0,6.0,Yes,A person runs into their pantry holding a bott...,cup;phone,A person runs in place while holding a bottle ...,c015 0.00 32.00;c107 0.00 32.00,31.38


In [0]:
ids=df["id"].values.tolist()
ind=[]
label=[]
data_feed_image_f=[]

In [0]:
len(ar)

12

In [0]:
data_feed_image.shape

(12, 250, 1280)

In [0]:
for i,v in enumerate(ar):
  try:
    ind.append(ids.index(v))
    data_feed_image_f.append(data_feed_image[i])
  except:
    pass

In [0]:
len(ind)

9

In [0]:
data_feed_image_f=np.asarray(data_feed_image_f)

In [0]:
data_feed_image_f.shape

(9, 250, 1280)

In [0]:
scripts=df['script'].values.tolist()

In [0]:
def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})

In [0]:
embed_fn = embed_useT('/content/drive/My Drive/USE/')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [0]:
sentence_encoding=embed_fn(scripts)

In [0]:
sentence_encoding.shape

(7985, 512)

In [0]:
data_feed_sentence=[]

In [0]:
for i in id:
  

In [0]:
vocab=set()
for i in df['script']:
  v=i.split()
  vocab.update(v)

In [0]:
vocab_len=len(vocab)

In [0]:
tok = Tokenizer(num_words=30000,lower=True)
tok.fit_on_texts(vocab)

In [0]:
seq_script=tok.texts_to_sequences(scripts)

In [0]:
max_seq_len=0
a=[]
for i in seq_script:
  a.append(len(i))
  if len(i)>max_seq_len:
    max_seq_len=len(i)

In [0]:
word2idx = tok.word_index

In [0]:
max_seq_len

84

In [0]:
mn=st.mean(a)

In [0]:
std=st.stdev(a)

In [0]:
thresh=int((mn+(3*std)))
thresh

43

In [0]:
for ind,val in enumerate(seq_script):
  seq_script[ind]=seq_script[ind][:thresh]

In [0]:
data_feed_script= pad_sequences(seq_script, maxlen=thresh+1)


In [0]:
data_feed_script.shape

(7985, 44)

In [0]:
emb={}
with open(os.path.join('/content/drive/My Drive/glove.6B.50d.txt')) as f :
  for l in f:
    v=l.split()
    w=v[0]
    emb[w]=v[1:]

In [0]:
t={}
emb_dim=50
num_words =vocab_len+1 
embedding_matrix = np.random.rand(num_words,emb_dim)*2-1
for word,i in word2idx.items():
  embedding_vector = emb.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = np.asarray(embedding_vector)
  else:
    t[word]=i
    embedding_matrix[i] = np.asarray(emb.get("unk"))
print(t)
print(len(t))

{"they're": 382, 'selfie': 449, "that's": 546, "it's": 627, 'selfies': 688, 'processds': 836, "couldn't": 852, "don't": 874, "weren't": 891, 'teddybear': 961, 'laudry': 1024, 'vegins': 1159, "didn't": 1181, "wouldn't": 1207, 'neaten': 1242, "won't": 1250, 'coffe': 1295, "bedroom's": 1385, "get's": 1392, 'quiddith': 1464, 'decines': 1550, "pantry's": 1577, 'begans': 1615, 'bathrom': 1628, 'refridgerator': 1636, "who's": 1652, "person's": 1658, "window's": 1679, "camera's": 1688, 'thens': 1723, "what's": 1728, 'theu': 1751, 'begings': 1789, 'doornobo': 1790, 'vacum': 1839, "house's": 1851, "fly's": 1874, "phone's": 1915, 'frig': 1941, 'visable': 1977, "can't": 1984, "aren't": 2003, 'bathrooom': 2009, 'enterway': 2025, "night's": 2080, "doesn't": 2109, "home's": 2123, 'sittiing': 2154, "they've": 2161, "door's": 2172, "anyone's": 2286, "clock's": 2294, 'smiliing': 2304}
53


In [0]:
classes=156

In [0]:
def network(text,act,sen):
  with tf.variable_scope('Action'):
    act_out=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences=False))(act)
  print(act_out.shape)

  with tf.variable_scope('Text'):
    text_out=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences=False))(text)  
  print(text_out.shape)  

  with tf.variable_scope('Sentence'):
    st=tf.keras.layers.Dense(250,activation='relu')(sen)
    st=tf.keras.layers.Dense(100,activation='relu')(st)

  fused_stacked_layer=tf.stack([act_out,text_out,st],axis=1)
  print(fused_stacked_layer.shape)

  fused_concat_layer=tf.keras.layers.Concatenate()([act_out,text_out,st])
  print(fused_concat_layer.shape)

  fused_atten_dense=tf.keras.layers.Dense(128,activation='relu')(fused_concat_layer)
  fused_atten_out=tf.keras.layers.Dense(3,activation='softmax')(fused_atten_dense)
  print(fused_atten_out.shape)

  fused_out_fin = tf.multiply(tf.expand_dims(fused_atten_out,2),fused_stacked_layer)
  print(fused_out_fin.shape)

  fused_out_fin = tf.reshape(fused_out_fin,[tf.shape(fused_out_fin)[0],fused_out_fin.shape[1]*fused_out_fin.shape[2]])
  print(fused_out_fin.shape)

  fused_fc=tf.keras.layers.Dense(170,activation='relu')(fused_out_fin)
  fused_fc_logits=tf.keras.layers.Dense(classes)(fused_fc)
  fused_fc_sft=tf.nn.softmax(fused_fc_logits)
  return fused_fc_logits,fused_fc_sft,fused_atten_out


In [0]:
sentence_encoding[:1]

In [0]:
ACTION=tf.placeholder(tf.float32,[None,20,1280])
TEXT=tf.placeholder(tf.int32,[None,thresh+1])
SENTENCE=tf.placeholder(tf.float32,[None,512])
Y=tf.placeholder(tf.int32,[None,1])

word_embeddings=tf.Variable(embedding_matrix,name="emb",trainable=False,dtype=tf.float32)
txt= tf.nn.embedding_lookup(word_embeddings,TEXT)

logit,prob,atten=network(txt,ACTION,SENTENCE)
one_hot=tf.one_hot(indices=Y,depth=classes,axis=-1)
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=one_hot,logits=logit))

with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
  optimizer=tf.train.AdamOptimizer(0.001).minimize(loss)

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for ep in range(1000):
    l,_=sess.run([loss,optimizer],feed_dict={ACTION:np.random.rand(5,20,1280),TEXT:data_feed_script[:5],Y:np.random.randint(152,size=(5,1)),SENTENCE:sentence_encoding[:5]})
    print(ep,":",l)

  

(?, 100)
(?, 100)
(?, 3, 100)
(?, 300)
(?, 3)
(?, 3, 100)
(?, 300)
0 : 5.084094
1 : 5.140191
2 : 5.0464463
3 : 5.066551
4 : 5.040895
5 : 5.055976
6 : 5.041914
7 : 5.057515
8 : 5.028638
9 : 5.0676007
10 : 5.0415964
11 : 5.093951
12 : 5.041438
13 : 5.1062355
14 : 5.0619154
15 : 5.048433
16 : 5.058339
17 : 5.0459924
18 : 5.0554686
19 : 5.047388
20 : 5.0548134
21 : 5.050895
22 : 5.0838923
23 : 5.020631
24 : 5.0613947
25 : 5.0646677
26 : 4.9761286
27 : 5.015681
28 : 5.0151606
29 : 5.091149
30 : 5.072516
31 : 5.1013336
32 : 5.1015444
33 : 5.0422206
34 : 5.0966477
35 : 5.0671477
36 : 5.086529
37 : 5.0568547
38 : 5.0729733
39 : 5.0722356
40 : 5.0251718
41 : 5.037053
42 : 5.062892
43 : 5.025443
44 : 5.062688
45 : 5.091768
46 : 5.0460234
47 : 5.07825
48 : 5.0746784
49 : 5.0552917
50 : 5.092469
51 : 5.049386
52 : 5.0131245
53 : 5.025819
54 : 4.996691
55 : 5.03922
56 : 5.0430665
57 : 5.079389
58 : 5.034032
59 : 5.0283127
60 : 5.070874
61 : 5.0709095
62 : 4.982889
63 : 5.076527
64 : 5.012154
65 : 5

KeyboardInterrupt: ignored